# 수화 동영상 파일 -> 텐서 파일(+ 미디어파이프 처리 영상 파일)

In [1]:
import os
import torch
import mediapipe as mp
import numpy as np
import cv2

In [2]:
# 모션 랜드마크를 리스트로 반환
def convert_landmark_to_tensor(landmarks, n_point):
    # 영상에 모션이 잡힐 경우 
    if landmarks: 
        motion_location = []
        for lm in landmarks.landmark:
            motion_location.append([lm.x, lm.y, lm.z])
        
        return motion_location
        
    # 영상에 모션이 잡히지 않을 경우
    else:
        return [[0] * 3 for _ in range(n_point)]
    

In [3]:
def make_tensor_and_video(input_video_path, output_tensor_path, output_video_path, video_save=False):
    
    # Prepare DrawingSpec
    mp_drawing = mp.solutions.drawing_utils 
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    # Config holistic
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(
        min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
    # 영상 가져오기
    cap = cv2.VideoCapture(input_video_path)

    # 영상 저장 1
    if video_save:
        fourcc = cv2.VideoWriter_fourcc(*'MP4V') # 영상 포맷
        out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (1280,720)) # 비디오 경로, 영상 포맷, 초당 프레임, width*height 

    # 영상 width, height 설정
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # 각 요소(왼손, 오른손, 얼굴, 포즈) 좌표 저장 리스트
    left_hand_list = []
    right_hand_list = []
    face_list = []
    pose_list = []

    while cap.isOpened():

        success, image = cap.read() 

        if not success: # 동영상 끝
            break

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = holistic.process(image)

        # Draw landmark annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
          image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(
          image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        
        # 각 랜드마크를 리스트로 변환 후 리스트에 저장
        left_hand_list.append(convert_landmark_to_tensor(results.left_hand_landmarks, 21))
        right_hand_list.append(convert_landmark_to_tensor(results.right_hand_landmarks, 21))
        face_list.append(convert_landmark_to_tensor(results.face_landmarks, 468))
        pose_list.append(convert_landmark_to_tensor(results.pose_landmarks, 33))
            
        # 영상 저장 2 (실질적인 영상 쓰기)
        if video_save:
            out.write(image)

    # 텐서로 변환
    left_hand_tensor = torch.FloatTensor(left_hand_list)
    right_hand_tensor = torch.FloatTensor(right_hand_list)
    face_tensor = torch.FloatTensor(face_list)
    pose_tensor = torch.FloatTensor(pose_list)
    
    # 텐서를 저장할 디렉토리 생성
    if not(os.path.isdir(output_tensor_path)):
        os.mkdir(output_tensor_path)
        
    # 텐서 파일 저장
    torch.save(left_hand_tensor, os.path.join(output_tensor_path, "left_hand.pt"))
    torch.save(right_hand_tensor, os.path.join(output_tensor_path, "right_hand.pt"))
    torch.save(face_tensor, os.path.join(output_tensor_path, "face.pt"))
    torch.save(pose_tensor, os.path.join(output_tensor_path, "pose.pt"))
    
    if video_save:
        # 동영사을 저장할 디렉토리 생성
        if not(os.path.isdir(output_video_path)):
            os.mkdir(output_video_path)
        
        # 영상 저장 3
        if video_save:
            out.release()

    holistic.close()
    cap.release()
    cv2.destroyAllWindows()

In [4]:
# 여러 디렉토리 생성 함수
def mkdirs(file_path_list: list, verbose=True):
    # ex) file_path_list= [".", "output", "images"]
    
    file_path = ""  # 생성할 디렉토리
    for path in file_path_list:
        file_path = os.path.join(file_path, path)
        
        # 디렉토리가 없다면 생성
        if not(os.path.isdir(file_path)):
            # 디렉토리 생성 안내 문구 출력
            if verbose:
                print("No {}".format(file_path))
                print("Make directory {}".format(file_path))
                print()
                
            # 디렉토리 생성
            os.mkdir(file_path)

In [5]:
def convert_videos_to_tensor(input_video_path_list=[".", "videos"],
                             output_tensor_path_list=[".", "output", "tensor"],
                             output_video_path_list=[".", "output", "video"],
                             videos_save=False):
    
    # 디렉토리 풀내임 생성
    input_video_path = os.path.join(*input_video_path_list)
    output_tensor_path = os.path.join(*output_tensor_path_list)
    output_video_path = os.path.join(*output_video_path_list)

    # 안내 문구 출력
    print("{:20}{}".format("intput_video_path: ", input_video_path))
    print("{:20}{}".format("output_tensor_path: ", output_tensor_path))
    print("{:20}{}".format("videos_save: ", str(videos_save)))
    if videos_save:
        print("{:20}{}".format("output_video_path: ", output_video_path))
    print()
    
    # 디렉토리 생성
    mkdirs(input_video_path_list)
    mkdirs(output_tensor_path_list)
    if videos_save:
        mkdirs(output_video_path_list)

    # 각 비디오 파일 
    videos = os.listdir(input_video_path)
    videos.sort()

    # 동영상의 모션을 텐서로 변환
    for i, video in enumerate(videos):
        make_tensor_and_video(os.path.join(input_video_path, video),
                              os.path.join(output_tensor_path, video[:-4]),  # [:-4]: 확장자 제거
                              os.path.join(output_video_path, video),
                              videos_save)
        if i%5==0:
            print("{}/{} videos completed".format(i, len(videos)))

In [6]:
# 동영상 개장 3~5초 정도 소요
# ex) 동영상 50개: 4분 소요
convert_videos_to_tensor()

intput_video_path:  ./videos
output_tensor_path: ./output/tensor
videos_save:        False

0/21 videos completed
5/21 videos completed
10/21 videos completed
15/21 videos completed
20/21 videos completed


In [8]:
# shape 확인
print(torch.load("./output/tensor/KETI_SL_0000000002/left_hand.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/right_hand.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/pose.pt").shape)
print(torch.load("./output/tensor/KETI_SL_0000000002/face.pt").shape)

torch.Size([160, 21, 3])
torch.Size([160, 21, 3])
torch.Size([160, 33, 3])
torch.Size([160, 468, 3])


In [10]:
# face 좌표 확인
pose_tensor = torch.load("./output/tensor/KETI_SL_0000000002/pose.pt")
print(pose_tensor)

tensor([[[ 0.4976,  0.2136, -0.8674],
         [ 0.5105,  0.1642, -0.8141],
         [ 0.5222,  0.1617, -0.8142],
         ...,
         [ 0.4254,  1.9055,  0.2465],
         [ 0.5679,  1.9551, -0.2476],
         [ 0.4383,  1.9436, -0.0123]],

        [[ 0.4934,  0.2121, -0.7493],
         [ 0.5069,  0.1636, -0.7004],
         [ 0.5184,  0.1612, -0.7005],
         ...,
         [ 0.4328,  1.8615,  0.1323],
         [ 0.5596,  1.9070, -0.2900],
         [ 0.4418,  1.9000, -0.1024]],

        [[ 0.4959,  0.2109, -0.7550],
         [ 0.5092,  0.1620, -0.7042],
         [ 0.5207,  0.1596, -0.7043],
         ...,
         [ 0.4285,  1.8531,  0.1467],
         [ 0.5592,  1.8996, -0.2816],
         [ 0.4397,  1.8897, -0.0702]],

        ...,

        [[ 0.4917,  0.2157, -0.8534],
         [ 0.5045,  0.1680, -0.8078],
         [ 0.5158,  0.1655, -0.8078],
         ...,
         [ 0.4347,  1.9247,  0.3189],
         [ 0.5718,  1.9957, -0.1753],
         [ 0.4467,  1.9968,  0.0181]],

        [[

현재 가장 바깥 리스트만 텐서로 바뀌고 텐서 내부는 전부 리스트임

---